In [ ]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from magenta.models.score2perf import score2perf
from tensor2tensor.data_generators import text_encoder
import note_seq

## Helper Functions

In [2]:
def decode(ids, encoder):
    ids = list(ids)
    if text_encoder.EOS_ID in ids:
        ids = ids[:ids.index(text_encoder.EOS_ID)]
    return encoder.decode(ids)

In [3]:
def load_midi(midi_file_path):
    return note_seq.midi_file_to_note_sequence(midi_file_path)

## Unconditioned Model

In [ ]:
class PianoPerformanceLanguageModelProblem(score2perf.Score2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

In [ ]:
problem = PianoPerformanceLanguageModelProblem()

In [ ]:
from tensor2tensor import problems
unconditional_encoders = problem.get_feature_encoders()

In [ ]:
from tensor2tensor.utils import trainer_lib
hparams = trainer_lib.create_hparams(hparams_set='transformer_tpu')
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

In [ ]:
from tensor2tensor.utils import decoding
decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

In [ ]:
run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    'transformer', hparams, run_config,
    decode_hparams=decode_hparams)

In [ ]:
ckpt_path = 'models/music_transformer/unconditional_model_16.ckpt'

In [ ]:
def input_generator():
    global targets
    global decode_length
    while True:
        print('yielding', len(targets))
        yield {
            'targets': np.array([targets], dtype=np.int32),
            'decode_length': np.array(decode_length, dtype=np.int32)
        }

In [ ]:
targets = []
decode_length = 0

In [ ]:
input_fn = decoding.make_input_fn_from_generator(input_generator())
unconditional_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

In [ ]:
_ = next(unconditional_samples)

In [ ]:
targets = []
decode_length = 1024
sample_ids = next(unconditional_samples)['outputs']

In [ ]:
midi_filename = decode(
    sample_ids,
    encoder=unconditional_encoders['targets'])

In [ ]:
unconditional_ns = note_seq.midi_file_to_note_sequence(midi_filename)

In [ ]:
import shutil
shutil.copyfile(midi_filename, 'output.midi')